# Building A Hello World Skill
This notebook demonstrates how to use the Cortex Python Libraries to build a simple Cortex Action and Skill.  For simple cases like this, the entire Skill can be defined, tested, and deployed right from a single notebook.

### Building a Cortex Action using iPython Magic
The Cortex Action is a self-contained code module that encapsulates a task - in other words, it is a function in computer science terms.  It can be a long running batch job, or stateful microservice.  In this "Hello World" example, you will create the "Hello World" Action leveraging the __%%cortex_action__ iPython cell magic to automatically deploy it to your Cortex environment as a daemon.

### Important information

Almost any function can be configured into a Cortex Action. 

Imagine the whole action as a sandwich. The the "meat" is the code or function you want to deploy (same as in any other environment) and Cortex is the bread.

Cortex Actions are the raw functionality you wish to use in your solutions.  Actions are designed in Cortex to be able to be decoupled from a particular Skill so that they may be reused in other Skills. 

Cortex Skills are the computational components of an Agent. A skill executes an atomic unit of work and can be triggered by one or more inputs to produce one or more outputs. Cortex Skills' inputs, outputs, and internal routing must be handled properly. 

    The inputs are taken as JSON objects, and the outputs are returned as JSON objects.

    These JSON objects can contain as many inputs or outputs as any normal function. 

To illustrate this process let's use the very basic example below and create the classic 'Hello World' example:

## Step 1: Define the function and create the action

In [ ]:
# Common Setup
%run setup.ipynb

In [ ]:
from cortex import Cortex
Cortex.login()

In [ ]:
%%cortex_action --name '<your_name>/hello-world' --function hello_world  --daemon 

from cortex import Cortex, Message

def hello_world(params):
    msg = Message(params)
    cortex = Cortex.client(api_endpoint=msg.apiEndpoint, token=msg.token)
    text = msg.payload.get('text')
    return cortex.message({'message': 'Hello %s!' % text}).to_params()

### Testing Actions
Using the Cortex client, lets test our Action to ensure it deployed properly.  

__PRO TIP__ Functionality testing should be done after every step of the Skill building process (either in a notebook or programatically)

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did above for '--name'
action = cortex.action('<YOURNAME>/hello-world')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _text_ parameter in the Message payload.

In [ ]:
from cortex import Cortex, Message
rs = action.invoke(cortex.message({'text': 'Cortex'}))
rs.payload

## Step 2. Build a Cortex Skill from the Hello World Action
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will pipe an Input to the Action defined above and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Hello World" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
hello_world = builder.skill('<YOURNAME>/hello-world').title('Hello World <YOURNAME>').description('Introductory Hello World Skill by <YOUR NAME>')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares which Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
hello_world = hello_world.input('say-hello').title('Your Name').parameter(name='text', type='string').all_routing(action, 'greeting').build()

In the previous step, we referenced an Output called **greeting**.  We can create that Output here using the Output sub-builder.

In [ ]:
hello_world = hello_world.output('greeting').title('Greeting').parameter(name='message', type='string').build()

## Step 3. Preview the CAMEL Document
We can preview the CAMEL (Cognitive Agent Modeling And Execution Language) document that each builder will create using the _to__camel_ method.

In [ ]:
hello_world.to_camel()

## Step 4. Final Build and Publish
You are now ready to build and publish your Skill to you Cortex Catalog.

In [ ]:
skill = hello_world.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='say-hello', message=cortex.message({'text': 'Cortex'}))
rs.payload

# Create Your Own Action

Now that you have seen some basic action and skill creation, it's time to try it on your own. Below is a basic code sample, and it's your job to turn it into a Cortex Action. 

Convert the included movie_maker function to a Cortex action, and then build a skill with that action.

### 1. Your task ##
Convert the following movie_maker function into a Cortex action and then build your own skill with that action.

HINT: Use the hello_world walkthrough as an example **(a complete example is included at the end of this notebook)**

### 2. Read this Important Information

You can import libraries that the action will need (even if you dont have the library installed on your own machine). If a library requires a **pip install**, you can add it to the 

docker container using cell magic with a **'--requirements'** flag. 

### 3. Examples and tips:
For example the 'seaborn' library, cannot be simply imported. It must be installed via pip first. To do this the cell magic up top would look like:

**'%%cortex_action --name '<namespace>/my-function' --function my_function --requirements seaborn' --daemon**

Any other additional libraries that must be installed in the container can be added to the requirements flag with a comma. (ex. '--requirements seaborn,nltk')

To create a daemon or job, simply add a '--daemon' or '--job' flag to the top of the cell magic.

**Note** For the movie_maker function, you will need to add the 'faker' library as a requirement

The movie_maker function takes in the name of a movie as a string, and outputs a dictionary with four cast members.

### 4. Try it now (A completed example is at the end of this notebook)

* It's helpful to take a look back at the 'hello_world' action to see what kind of things Cortex needs to turn a function into an action

* Remember, Cortex actions are deployed as docker images, so make sure to add in the code for the Cortex Magics

* Input and Output are passed as JSON objects in Cortex, so rather than explicitly passing an object to a function within your action like 
  'multiplier(x,y)', you would pass it as 'multiplier(params)' and then retrieve the objects x and y out of the params JSON object
 
* Rather than returning an object, you must call the cortex Message builder to return the message as a payload. This is a JSON structure such as '{'key' : object}'. 

* Remember, this movie_maker function requires a library called 'faker'. This means it must be added in the '--requirements' flag up top with the magics



In [ ]:
#Hint the cell magic goes first

#Here is the basic function that will be in your action (container)

#Don't forget to import what your action will need

#Remember Cortex expects messages to be passed in a certain format

from faker import Faker
def movie_maker(name_of_movie):
    fake = Faker()
    movie = {'name_of_movie': name_of_movie, 'lead_1': fake.name(), 'lead_2': fake.name(), 'support_1': fake.name(), 'support_2': fake.name()}
    return movie

## Test the movie_maker action

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did in the action header
action = cortex.action('<YOURNAME>/<movie_maker_YOUR_INITIALS>')

In [ ]:
from cortex import Message
rs = action.invoke(cortex.message({'name_of_movie': 'Our Big Adventure'}))
movie_maker_output = rs.payload
movie_maker_output

# Creating the movie_maker Skill

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Hello World" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('<namespace>/<movie_maker_YOUR_INITIALS>').title('Movie Maker-<YOUR_INITIALS>').description('Generates a fake cast for a movie by <YOUR_INITIALS>')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares which Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('make-movie').title('Name of Movie').parameter(name='name_of_movie', type='string').all_routing(action, 'movie').build()

In the previous step, we referenced an Output called **movie**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('movie').title('Movie').parameter(name='movie', type='object', required=True).parameter(name='role', type='string', required=True).parameter(name='new_cast', type='string', required=True).build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to__camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='make-movie', message=cortex.message({'name_of_movie': 'Neils Big Adventure'}))
rs.payload

## Completed example

Expand the following to see how to make this action

In [ ]:
%%cortex_action --name '<YOURNAME>/<movie_maker_YOUR_INITIALS>' --function movie_maker --requirements faker --daemon

from faker import Faker
from cortex import Cortex, Message


def movie_maker(params):
    msg = Message(params)
    cortex = Cortex.client(api_endpoint=msg.apiEndpoint, token=msg.token)
    fake = Faker()
    name_of_movie = msg.payload.get('name_of_movie')
    movie = {'name_of_movie': name_of_movie, 'lead_1': fake.name(), 'lead_2': fake.name(), 'support_1': fake.name(), 'support_2': fake.name()}
    return cortex.message({'movie': movie}).to_params()

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did in the action header
action = cortex.action('<YOURNAME>/<movie_maker_YOUR_INITIALS>')

In [ ]:
action.get_deployment_status()

In [ ]:
from cortex import Message
rs = action.invoke(cortex.message({'name_of_movie': 'The Princess Bride 2'}))
movie_maker_output = rs.payload
movie_maker_output